# Rumination prediction - averaged participants' epochs

### Vectorization with ICA

### Imports

In [ ]:
%load_ext lab_black
import os
import pickle
from time import time
import pywt
import mne
import scipy
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import cesium.featurize
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, FloatSlider, interact
from sklearn.decomposition import FastICA
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

import sys

sys.path.append("..")
from utils import *

### Loading data

Loading EEG data and data from rumination questionnaire. By default create_df_data loads all info from given file but one can specify it by passing a list of desired labels from csv file.

In [ ]:
tmin, tmax = -0.1, 0.6
signal_frequency = 256
ERROR = 0
CORRECT = 1
random_state = 0

In [ ]:
channels_order_list = [
    "Fp1",
    "AF7",
    "AF3",
    "F1",
    "F3",
    "F5",
    "F7",
    "FT7",
    "FC5",
    "FC3",
    "FC1",
    "C1",
    "C3",
    "C5",
    "T7",
    "TP7",
    "CP5",
    "CP3",
    "CP1",
    "P1",
    "P3",
    "P5",
    "P7",
    "P9",
    "PO7",
    "PO3",
    "O1",
    "Iz",
    "Oz",
    "POz",
    "Pz",
    "CPz",
    "Fpz",
    "Fp2",
    "AF8",
    "AF4",
    "AFz",
    "Fz",
    "F2",
    "F4",
    "F6",
    "F8",
    "FT8",
    "FC6",
    "FC4",
    "FC2",
    "FCz",
    "Cz",
    "C2",
    "C4",
    "C6",
    "T8",
    "TP8",
    "CP6",
    "CP4",
    "CP2",
    "P2",
    "P4",
    "P6",
    "P8",
    "P10",
    "PO8",
    "PO4",
    "O2",
]

In [ ]:
channels_dict = dict(zip(channels_order_list, np.arange(1, 64, 1)))

In [ ]:
df_name = "go_nogo_df_mean"
pickled_data_filename = "../../data/" + df_name + ".pkl"
info_filename = "../../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_df.name = df_name
    # save loaded data into a pickle file
    epochs_df.to_pickle("../../data/" + epochs_df.name + ".pkl")
    print("Done. Pickle file created")

#### Average participants' error and correct epochs

In [ ]:
averaged_epochs_df = (
    epochs_df.groupby(
        ["id", "marker"],
        sort=False,
    )
    .apply(
        lambda group_df: pd.Series(
            {
                "epoch": np.mean(group_df["epoch"]),
                "Rumination Full Scale": np.mean(group_df["Rumination Full Scale"]),
            }
        )
    )
    .reset_index()
)

## Training and predictions

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from tempfile import mkdtemp


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import warnings

warnings.filterwarnings("ignore")


import numpy as np
import scipy.stats

- Computes ICA and then at each channel computes CWT (ica_n_components = N).
- For each band (frequency) from CWT set it computes features given in feature_dict parameter (eg. std or mean).
- Then it computes PCA on flattened ICA channels and features (outer_components = N)
- Ending feature vector has shape: outer_components from (ica_n_components * len(feature_dict) * frequencies)

#### Standard features for EEG analysis provided by Guo et al. (2012)

In [ ]:
def std_signal(t, m, e):
    return np.std(m)


def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))


def mean_energy_signal(t, m, e):
    return np.mean(m ** 2)


def skew_signal(t, m, e):
    return scipy.stats.skew(m)


def mean_signal(t, m, e):
    return np.mean(m)

#### Additional score functions 

The adjusted R-squared increases only if the new term improves the model more than would be expected by chance. It decreases when a predictor improves the model by less than expected by chance.

https://www.statisticshowto.com/adjusted-r2/

In [ ]:
def r2_adjusted_scorer(y_test, y_pred, p, n):
    r2 = r2_score(y_test, y_pred)
    r2_adj = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    return r2_adj

### Regressions grid search

Pipeline allows manipulation of vectorization's parameters. Base_steps dictionary consists of all steps of vectorization including standarization of data.

In rate_regression function, using GridSearchCV, cross-validation splitting strategy can be specified. Default cv = 5.
Results of cross-validated search are in **grid_search.cv_results** and chosen model is in **grid_search.best_estimator_**

In [ ]:
dataset = ERROR
dataset_name = "correct" if dataset == CORRECT else "error"

In [ ]:
X_train = np.array(
    averaged_epochs_df[averaged_epochs_df["marker"] == dataset]["epoch"].to_list()
)
y_train = np.array(
    averaged_epochs_df[averaged_epochs_df["marker"] == dataset][
        "Rumination Full Scale"
    ].to_list()
)

In [ ]:
X_test = []
y_test = []

#### Defined data transformers - custom data transformation steps

In [ ]:
def ChannelExtractionTransformer(channel_list):
    def transform(X):
        epochs_per_channels = np.transpose(X, (1, 0, 2))
        epochs_per_selected_channels = []

        for channel in channel_list:
            this_data = epochs_per_channels[channel]
            epochs_per_selected_channels.append(this_data)

        epochs_per_selected_channels = np.array(epochs_per_selected_channels)
        selected_channels_per_epoch = np.transpose(
            epochs_per_selected_channels, (1, 0, 2)
        )
        #         print(f"EXTRACTION {selected_channels_per_epoch.shape}")
        return selected_channels_per_epoch

    return FunctionTransformer(func=transform)


# swap channels and epochs axes: from epoch_channel_timepoints to channel_epoch_timepoints and vice versa
def ChannelDataSwap():
    def transform(X):
        data_channel_swaped = np.transpose(X, (1, 0, 2))
        return data_channel_swaped

    return FunctionTransformer(func=transform)


def IcaPreprocessingTransformer():
    def transform(X):
        timepoints_per_channel = np.concatenate(X, axis=1)
        return timepoints_per_channel.T

    return FunctionTransformer(func=transform)


def IcaPostprocessingTransformer(timepoints_count):
    def transform(X):
        X_ica_transposed = X.T
        ica_n_components = X.shape[1]

        epochs_count = int(X_ica_transposed.shape[1] / timepoints_count)
        data_per_channel = X_ica_transposed.reshape(
            ica_n_components, epochs_count, timepoints_count
        )
        return data_per_channel

    return FunctionTransformer(func=transform)


def CwtVectorizer(mwt="morl", cwt_density=2, cwt_octaves=6):
    def transform(X):
        cwt_per_channel = []
        for data in X:
            data_cwt = np.array(
                [cwt(epoch, mwt, cwt_density, octaves=6) for epoch in data]
            )
            cwt_per_channel.append(data_cwt)
        cwt_per_channel = np.array(cwt_per_channel)
        return cwt_per_channel

    return FunctionTransformer(func=transform)


def BinTransformer(step):
    def bin_epoch(epoch):
        new_channels = []
        for channel in epoch:
            bins_channel = []
            index = 0
            while index + step < len(channel):
                this_bin = np.mean(channel[index : index + step])
                bins_channel.append(this_bin)
                index += step
            new_channels.append(bins_channel)
        return new_channels

    def transform(X):
        binned_data = np.array([bin_epoch(epoch) for epoch in X])
        return binned_data

    return FunctionTransformer(func=transform)


def CwtFeatureVectorizer(feature_dict):
    def transform(X):
        vectorized_data = []

        for data_cwt in X:
            # cesium functions
            feature_set_cwt = cesium.featurize.featurize_time_series(
                times=None,
                values=data_cwt,
                errors=None,
                features_to_use=list(feature_dict.keys()),
                custom_functions=feature_dict,
            )
            features_per_epoch = feature_set_cwt.to_numpy()
            vectorized_data.append(features_per_epoch)
        vectorized_data = np.array(vectorized_data)
        return vectorized_data

    return FunctionTransformer(func=transform)


# transforms energy of each sub-band into relative energy of sub-band
def RelativeEnergyTransformer():
    def transform(X):
        vectorized_data = []

        for epoch in X:
            total_energy_of_epoch = np.sum(epoch)
            sub_band_relative_energies = np.array(
                [(sub_band_energy / total_energy_of_epoch) for sub_band_energy in epoch]
            )
            vectorized_data.append(sub_band_relative_energies)

        vectorized_data = np.array(vectorized_data)
        return vectorized_data

    return FunctionTransformer(func=transform)


# reshape data from (channels x epoch x features) to (epochs x channles x features)
# and then flatten it to (epoch x channels*features)
def PostprocessingTransformer():
    def transform(X):
        vectorized_data = np.stack(X, axis=1)
        epochs_per_channel_feature = vectorized_data.reshape(
            vectorized_data.shape[0], -1
        )
        return epochs_per_channel_feature

    return FunctionTransformer(func=transform)

# Experiments

Prepare df for results:

In [ ]:
results_df = pd.DataFrame()

Define regression models:

In [ ]:
knn = ("knn", KNeighborsRegressor())
knn_params = dict(
    knn__n_neighbors=np.arange(8, 28, 3),
)

gbr = ("gbr", GradientBoostingRegressor())
gbr_params = dict(
    gbr__n_estimators=np.arange(1, 40, 5),
)

lasso = ("lasso", Lasso())
lasso_params = dict(lasso__alpha=np.arange(0.1, 0.5, 0.1))

svr = ("svr", SVR())
svr_params = dict(
    svr__kernel=["rbf", "linear", "sigmoid"],
    svr__C=[0.01, 0.1, 1],
)

In [ ]:
def rate_regression(
    X_train, y_train, X_test, y_test, regressor, regressor_params, base_steps, cv=5
):
    pipeline = Pipeline(steps=base_steps + [regressor])
    param_grid = regressor_params
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv,
        scoring={"r2", "neg_mean_absolute_error"},
        refit="r2",
        n_jobs=10,
        verbose=10,
    )
    grid_search.fit(X_train, y_train)

    return grid_search

Calculate p-value 

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv


def calculate_p(estimator, X_train, y_train):
    baseline_estimator = DummyRegressor(strategy="mean")

    # check if difference between algorithms is real
    t, p = paired_ttest_5x2cv(
        estimator1=baseline_estimator,
        estimator2=estimator,
        X=X_train,
        y=y_train,
        scoring="r2",
        random_seed=0,
    )

    # summarize
    print(f"     The P-value is = {p:.3f}")
    print(f"     The t-statistics is = {t:.3f}")

    return t, p

In [ ]:
def run_experiment(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    results_df,
    function_name="-",
):

    for (regressor, params) in tested_regressors:
        print(f"Rating {regressor} \n")
        tested_params = {**regressor_params, **params}

        grid_result = rate_regression(
            X_train, y_train, X_test, y_test, regressor, tested_params, base_steps, cv=2
        )

        #     predictions = grid_result.predict(X_test)
        #     r2 = grid_result.score(X_test, y_test)
        #     mae = mean_absolute_error(y_test, predictions)
        #     r2_adj = r2_adjusted_scorer(y_test, predictions, len(X_test[0]), len(X_test))

        best_estimator_index = grid_result.best_index_
        mean_cv_r2 = grid_result.cv_results_["mean_test_r2"][best_estimator_index]
        std_cv_r2 = grid_result.cv_results_["std_test_r2"][best_estimator_index]
        mean_cv_neg_mean_absolute_error = grid_result.cv_results_[
            "mean_test_neg_mean_absolute_error"
        ][best_estimator_index]
        std_cv_neg_mean_absolute_error = grid_result.cv_results_[
            "std_test_neg_mean_absolute_error"
        ][best_estimator_index]

        print(f"     Best parameters: {grid_result.best_params_}")
        print(f"     mean r2: {mean_cv_r2}           ± {round(std_cv_r2,3)}\n")

        cv_results = grid_result.cv_results_
        t_statistics, p_value = calculate_p(
            grid_result.best_estimator_, X_train, y_train
        )

        data = {
            "data_set": dataset_name,
            "pipeline_name": pipeline_name + "-" + function_name,
            "function": function_name,
            "model": regressor[0],
            "parameters": grid_result.best_params_,
            "mean_cv_r2": mean_cv_r2,
            "std_cv_r2": std_cv_r2,
            "mean_cv_mae": mean_cv_neg_mean_absolute_error,
            "std_cv_mae": std_cv_neg_mean_absolute_error,
            "cv_results": cv_results,
            "p-value": p_value,
            "t-stats": t_statistics,
            "best_estimator": grid_result.best_estimator_,
        }

        results_df = results_df.append(data, ignore_index=True)
    return results_df

Define significant channels - rest will be excluded

In [ ]:
red_box = [
    "F1",
    "Fz",
    "F2",
    "FC1",
    "FCz",
    "FC2",
    "C1",
    "Cz",
    "C2",
    "CP1",
    "CPz",
    "CP2",
    "P1",
    "Pz",
    "P2",
]
significant_channels = [channels_dict[channel] for channel in red_box]

### Experiment 1
- Models: KNN, GBR, Lasso, SVR
- vectorize with ICA-cwt-PCA

In [ ]:
pipeline_name = "ICA_cut_cwt"

In [ ]:
regressor_params = dict(
    ica__n_components=np.arange(3, 16, 2),
    pca__n_components=np.arange(4, 31, 2),
)

In [ ]:
tested_regressors = [
    (lasso, lasso_params),
    (gbr, gbr_params),
    (knn, knn_params),
    (svr, svr_params),
]

In [ ]:
base_steps = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    #     ("channel_data_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state)),
]

Run experiment:

In [ ]:
results_df = run_experiment(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    results_df,
)

In [ ]:
results_df

In [ ]:
results_df.to_pickle("../../data/regression_ICA_cut_" + dataset_name + ".pkl")

### Experiment 2

- Models: KNN, GBR, Lasso, SVR
- vectorize with ICA-bins-PCA

In [ ]:
step_in_ms = 50
step_tp = int(signal_frequency * step_in_ms / 1000)

In [ ]:
pipeline_name = "ICA_cut_bins"

In [ ]:
regressor_params = dict(
    ica__n_components=np.arange(3, 16, 2),
    pca__n_components=np.arange(4, 31, 2),
)

In [ ]:
tested_regressors = [
    (lasso, lasso_params),
    (gbr, gbr_params),
    (knn, knn_params),
    (svr, svr_params),
]

In [ ]:
base_steps = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state)),
]

Run experiment:

In [ ]:
results_df = run_experiment(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    results_df,
)

In [ ]:
results_df

In [ ]:
results_df.to_pickle("../../data/regression_ICA_cut_" + dataset_name + ".pkl")

### Experiment 2'
- Models: KNN, GBR, Lasso, SVR
- vectorize with ICA-PCA

In [ ]:
pipeline_name = "ICA_cut"

In [ ]:
regressor_params = dict(
    ica__n_components=np.arange(3, 16, 2),
    pca__n_components=np.arange(4, 31, 2),
)

In [ ]:
tested_regressors = [
    (lasso, lasso_params),
    (gbr, gbr_params),
    (knn, knn_params),
    (svr, svr_params),
]

In [ ]:
base_steps = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    #     ("channel_data_swap", ChannelDataSwap()),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state)),
]

In [ ]:
results_df = run_experiment(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    results_df,
)

In [ ]:
results_df

### Experiment 3

- Models: KNN, GBR, Lasso, SVR
- vectorize with ICA-bins-cwt-PCA

In [ ]:
pipeline_name = "ICA_cut_bins_cwt"

In [ ]:
regressor_params = dict(
    ica__n_components=np.arange(3, 16, 2),
    pca__n_components=np.arange(4, 31, 2),
)

In [ ]:
tested_regressors = [
    (lasso, lasso_params),
    (gbr, gbr_params),
    (knn, knn_params),
    (svr, svr_params),
]

In [ ]:
base_steps = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state)),
]

In [ ]:
results_df = run_experiment(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    results_df,
)

In [ ]:
results_df

In [ ]:
results_df.to_pickle("../../data/regression_ICA_cut_" + dataset_name + ".pkl")

In [ ]:
results_df.to_csv("../../data/regression_ICA_cut_" + dataset_name + ".csv")

### Experiment 4
- Models: KNN, GBR, Lasso, SVR
- ICA-cwt-functions

In [ ]:
pipeline_name = "ICA_cut_function"

In [ ]:
guo_features = [
    {"std": std_signal},
    {"abs_diffs": abs_diffs_signal},
    {"energy": mean_energy_signal},
    #     {"skew": skew_signal},
    {"mean": mean_signal},
]

In [ ]:
regressor_params = dict(
    ica__n_components=np.arange(3, 16, 2),
    pca__n_components=np.arange(4, 31, 2),
)

In [ ]:
tested_regressors = [
    (lasso, lasso_params),
    (gbr, gbr_params),
    (knn, knn_params),
    (svr, svr_params),
]

In [ ]:
for feature_function_dict in guo_features:
    print(f"Featurize with {feature_function_dict.keys()} function")

    # define base steps
    this_base_steps = [
        ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
        ("ica_preprocessing", IcaPreprocessingTransformer()),
        ("ica", FastICA(random_state=random_state)),
        (
            "ica_postprocessing",
            IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
        ),
        ("cwt", CwtVectorizer()),
        ("cwt_feature", CwtFeatureVectorizer(feature_dict=feature_function_dict)),
        ("postprocessing", PostprocessingTransformer()),
        ("scaler", StandardScaler()),
        ("pca", PCA(random_state=random_state)),
    ]

    # rate different models
    results_df = run_experiment(
        tested_regressors,
        regressor_params,
        pipeline_name,
        X_train,
        X_test,
        y_train,
        y_test,
        dataset_name,
        this_base_steps,
        results_df,
        function_name=list(feature_function_dict.keys())[0],
    )

In [ ]:
results_df.to_pickle("../../data/regression_ICA_cut_" + dataset_name + ".pkl")

### Experiment 5
- Models: KNN, GBR, Lasso, SVR
- ICA-bins-cwt-functions

In [ ]:
pipeline_name = "ICA_cut_bins_function"

In [ ]:
guo_features = [
    {"std": std_signal},
    {"abs_diffs": abs_diffs_signal},
    {"energy": mean_energy_signal},
    #     {"skew": skew_signal},
    {"mean": mean_signal},
]

In [ ]:
regressor_params = dict(
    ica__n_components=np.arange(3, 16, 2),
    pca__n_components=np.arange(4, 31, 2),
)

In [ ]:
tested_regressors = [
    (lasso, lasso_params),
    (gbr, gbr_params),
    (knn, knn_params),
    (svr, svr_params),
]

In [ ]:
for feature_function_dict in guo_features:
    print(f"Featurize with {feature_function_dict.keys()} function")

    # define base steps
    this_base_steps = [
        ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
        ("ica_preprocessing", IcaPreprocessingTransformer()),
        ("ica", FastICA(random_state=random_state)),
        (
            "ica_postprocessing",
            IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
        ),
        ("channel_data_swap", ChannelDataSwap()),
        ("binning", BinTransformer(step=step_tp)),
        ("data_channel_swap", ChannelDataSwap()),
        ("cwt", CwtVectorizer()),
        ("cwt_feature", CwtFeatureVectorizer(feature_dict=feature_function_dict)),
        ("postprocessing", PostprocessingTransformer()),
        ("scaler", StandardScaler()),
        ("pca", PCA(random_state=random_state)),
    ]

    # rate different models
    results_df = run_experiment(
        tested_regressors,
        regressor_params,
        pipeline_name,
        X_train,
        X_test,
        y_train,
        y_test,
        dataset_name,
        this_base_steps,
        results_df,
        function_name=list(feature_function_dict.keys())[0],
    )

In [ ]:
# with pd.option_context("display.max_colwidth", -1):
#     display(results_df)

In [ ]:
results_df.to_pickle("../../data/regression_ICA_cut_" + dataset_name + ".pkl")

In [ ]:
results_df.to_csv("../../data/regression_ICA_cut_" + dataset_name + ".csv")

#### Estimator 1

In [ ]:
# The P-value is = 0.038
# The t-statistics is = 2.796

steps = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=3)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    #     ("channel_data_swap", ChannelDataSwap()),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=26)),
    ("gbr", GradientBoostingRegressor(n_estimators=31)),
]

estimator1 = Pipeline(steps=steps)

#### Estimator 2

In [ ]:
# The P-value is = 0.164
# The t-statistics is = 1.630

steps2 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=7)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    #     ("channel_data_swap", ChannelDataSwap()),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=14)),
    ("svr", SVR(C=1, kernel="rbf")),
]

estimator2 = Pipeline(steps=steps2)

#### Estimator 3

In [ ]:
# The P-value is = 0.464
# The t-statistics is = 0.793

steps3 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=11)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"std": std_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=22)),
    ("gbr", GradientBoostingRegressor(n_estimators=21)),
]

estimator3 = Pipeline(steps=steps3)

#### Estimator 4

In [ ]:
# The P-value is = 0.184
# The t-statistics is = -1.542

steps4 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=15)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"std": std_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=12)),
    ("knn", KNeighborsRegressor(n_neighbors=8)),
]
estimator4 = Pipeline(steps=steps4)

#### Estimator 5

In [ ]:
# The P-value is = 0.766
# The t-statistics is = 0.314

steps5 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=15)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"std": std_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=16)),
    ("svr", SVR(C=1, kernel="rbf")),
]

estimator5 = Pipeline(steps=steps5)

In [ ]:
# The P-value is = 0.319
# The t-statistics is = 1.107

steps6 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=15)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"abs_diffs": abs_diffs_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=20)),
    ("svr", SVR(C=1, kernel="rbf")),
]

estimator6 = Pipeline(steps=steps6)

In [ ]:
# The P-value is = 0.922
# The t-statistics is = -0.103

steps7 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=15)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"energy": mean_energy_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=18)),
    ("svr", SVR(C=1, kernel="rbf")),
]

estimator7 = Pipeline(steps=steps7)

In [ ]:
# The P-value is = 0.037
# The t-statistics is = 2.825

steps8 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=15)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("channel_data_swap", ChannelDataSwap()),
    ("binning", BinTransformer(step=step_tp)),
    ("data_channel_swap", ChannelDataSwap()),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"energy": mean_energy_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=4)),
    ("knn", KNeighborsRegressor(n_neighbors=8)),
]
estimator8 = Pipeline(steps=steps8)

In [ ]:
# The P-value is = 0.062
# The t-statistics is = 2.400

steps9 = [
    ("channels_filtering", ChannelExtractionTransformer(significant_channels)),
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=random_state, n_components=11)),
    (
        "ica_postprocessing",
        IcaPostprocessingTransformer(timepoints_count=X_train.shape[-1]),
    ),
    ("cwt", CwtVectorizer()),
    ("cwt_feature", CwtFeatureVectorizer(feature_dict={"energy": mean_energy_signal})),
    ("postprocessing", PostprocessingTransformer()),
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=random_state, n_components=10)),
    ("knn", KNeighborsRegressor(n_neighbors=8)),
]

estimator9 = Pipeline(steps=steps9)

## 5 by 2 CV using MLxtend package

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv

# check if difference between algorithms is real
t, p = paired_ttest_5x2cv(
    estimator1=baseline_estimator,
    estimator2=estimator6,
    X=X_train,
    y=y_train,
    scoring="r2",
    random_seed=0,
)

# summarize
print(f"The P-value is = {p:.3f}")
print(f"The t-statistics is = {t:.3f}")
# interpret the result
if p <= 0.05:
    print(
        "Since p<0.05, We can reject the null-hypothesis that both models perform equally well on this dataset. We may conclude that the two algorithms are significantly different."
    )
else:
    print(
        "Since p>0.05, we cannot reject the null hypothesis and may conclude that the performance of the two algorithms is not significantly different."
    )